In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("chest_xray"))

from os import listdir, makedirs
from os.path import join, exists, expanduser

from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import tensorflow as tf
from keras.utils.data_utils import Sequence
import sys
from PIL import *
sys.modules['Image'] = Image 
print(os.listdir("chest_xray/train"))


['test', 'train', 'val']


Using TensorFlow backend.


['NORMAL', 'PNEUMONIA']


In [2]:
img_width, img_height = 128,128
train_data = 'chest_xray/train'
test_data = 'chest_xray/test'
val_data = 'chest_xray/val'

In [3]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)
val_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data,
    target_size=(128,128),
    batch_size=16,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_data,
    target_size=(128,128),
    batch_size=16,
    class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
    val_data,
    target_size=(128,128),
    batch_size=16,
    class_mode='categorical')

Found 5216 images belonging to 2 classes.
Found 524 images belonging to 2 classes.
Found 116 images belonging to 2 classes.


In [7]:
#import inception with pre-trained weights. do not include fully #connected layers
from keras.applications.resnet34 import ResNet34
model = ResNet34(weights='imagenet', include_top=False)

ModuleNotFoundError: No module named 'keras.applications.resnet34'

In [ ]:
result = model.output
result = GlobalAveragePooling2D()(result)
# add a fully-connected layer
# result1 = Dense(256, activation='relu')(result)
result = Dense(512, activation='relu')(result)
result = Dense(256, activation='relu')(result)
# and a fully connected output/classification layer
predictions = Dense(2, activation='sigmoid')(result)

In [ ]:
inception_transfer = Model(inputs=model.input, outputs=predictions)


In [ ]:
inception_transfer.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

In [ ]:
history_pretrained = inception_transfer.fit_generator(
    train_generator,
    epochs=16, shuffle = True, verbose = 1, validation_data = test_generator)